## 초기 system prompt 설정 및 벡터 store 생성

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from random import randint, choice

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from openai import OpenAI
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

permission = "cto"

# 벡터 DB 세팅
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embedding_function=embedding_model)

C:\Users\miny0\AppData\Local\Temp\ipykernel_26324\979826118.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
C:\Users\miny0\AppData\Local\Temp\ipykernel_26324\979826118.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embeddi

## 초기 질문 리스트 가져오기

In [2]:
import json

def load_questions_from_jsonl_minimal(path: str, key: str = "question"):
    """JSONL에서 각 라인의 `question` 값만 순서/중복/공백 그대로 추출"""
    questions = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            obj = json.loads(line)
            if isinstance(obj, dict) and key in obj:
                questions.append(obj[key])
    return questions

# 사용 예시
initial_questions = load_questions_from_jsonl_minimal(f"./company_questions_{permission}.jsonl")
initial_questions

['내부 평가 피드백 문서에서 프론트엔드팀의 업무 수행 능력에 대한 성과는 어떤가요?',
 '프론트엔드팀의 팀워크 및 커뮤니케이션 부분에서 발생한 의견 충돌에 대한 구체적인 사례는 무엇인가요?',
 '기술적 기여도 평가에서 A/B 테스트 도입이 긍정적인 결과를 가져온 사례에 대해 자세히 설명해 주실 수 있나요?',
 '개선 포인트 중 팀 내 갈등 해결을 위한 커뮤니케이션 교육의 필요성에 대한 구체적인 이유는 무엇인가요?',
 "문제 해결 능력 강화를 위한 후속 조치 제안 중 '문제 해결 세션'의 구체적인 진행 방식은 어떻게 되나요?",
 'CTO 의사결정 참고 사항에서 언급된 리스크 관리의 필요성에 대해 좀 더 자세히 설명해 주실 수 있나요?',
 '프론트엔드팀의 기술 혁신을 위한 자원 투자 필요성에 대한 구체적인 예시는 무엇인가요?',
 '성과 기반 평가 도입 검토에 대한 구체적인 계획이나 방향성이 있나요?',
 '내부 평가 피드백 문서에서 제시된 정기적인 워크숍의 목표와 기대 효과는 무엇인가요?',
 '프론트엔드팀의 문제 해결 능력에서 개선이 필요한 부분은 어떤 것들이 있으며, 이를 어떻게 해결할 계획인가요?',
 '2025년 1분기부터 3분기까지의 프로젝트에서 프론트엔드팀의 주요 성과는 무엇이었나요?',
 '내부 평가 피드백 문서에서 제시된 피드백 루프 구축의 구체적인 실행 방안은 무엇인가요?',
 "프론트엔드팀의 기술적 기여도 평가에서 '기술적 문제 발생 시 신속하게 대응'의 구체적인 사례는 무엇인가요?",
 '팀워크 및 커뮤니케이션 부분에서 정기 회의를 통해 의견을 교환한 방식은 어떤 것이었나요?',
 "프론트엔드팀의 근무 태도 평가에서 '코드 품질 개선'에 대한 구체적인 지표는 무엇인가요?",
 '내부 평가 피드백 문서에서 제안된 외부 강사 초빙의 필요성에 대한 근거는 무엇인가요?',
 "프론트엔드팀의 기술적 기여도 평가에서 '기존 시스템 개선'의 구체적인 사례는 무엇인가요?",
 '팀 내 협업이 원활하였다는 평가의 구체적인 근거는 무엇인가요?',


In [3]:
def question_chain_setting():
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)

    basic_prompt = PromptTemplate.from_template(
        """
            다음 제공되는 대화 내역을 기반으로 다음에 나올 수 있는 사용자가 궁금해할 만한 후속 질문 하나를 사용자 입장에서 질문하세요.

            대화 내역: {history}

            절대로 AI입장에서의 질문을 만들지 마세요.
            ex) 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다.
            ex) 추가적으로 궁금한 사항이 있으신가요?
            ex) 데이터 접근 권한 관리 방침에 대해 더 궁금한 사항이 있으면 말씀해 주세요.
            ex) 안녕하세요. 다른 궁금한 점이 있으신가요?
            ex) 안녕하십니까? 도움이 필요하신 부분이 있으신가요?
            ex) ~~ 예시는 다음과 같습니다.


            답변형태는 다음과 같이 해주세요.
            ex) 빌드 결과물의 확인 방법이나 배포 과정에 대해 알고 싶어.
            ex) 문서화 시 유의사항 알려줘.
            ex) 빌드/배포 가이드의 '6. 문서화' 파트에서 문서화해야 할 내용은 뭐야?
            ex) 모델 개발 & 성능 보고서에서 사용된 프로그래밍 언어는 무엇이야?

            <추가 지시사항>
            이전에 나왔던 질문은 최대한 만들지 말고, 연결된 질문으로 조금이라도 다른 질문을 만들어주세요.

        """
    )

    question_chain = basic_prompt | llm | StrOutputParser()

    return question_chain

In [4]:
question_chain = question_chain_setting()

In [5]:
# tool_prompt = f"사용자는 {permission}팀이며 반드시 {permission}_search 툴을 호출하세요."
tool_prompt = f"사용자는 {permission}로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""
당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 정중하고 사문적인 어조로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 정중하고 사무적인 어조로 답변해야 합니다.
8. {tool_prompt}
"""

SYSTEM_PROMPT = {
    "role": "system",
    "content": system_message
}

def generate_one_chat_sample(initial_q: str, max_turns: int = 5):
    messages = [SYSTEM_PROMPT]
    current_q = initial_q
    for turn in range(randint(2, max_turns + 1)):
        # (1) User 질문 추가
        messages.append({"role": "user", "content": current_q})
        print(current_q)

        tool_call = f"<tool_call>{{\"name\": \"{permission}_search\", \"arguments\": {{\"keyword\": \"{current_q}\"}}}}</tool_call>"

        messages.append({"role": "assistant", "content": tool_call})

        # (2) 문서 검색 (vectorstore에서)
        docs = vectorstore.similarity_search(current_q, k=4)
        ref_text = "\n".join([f"{doc.page_content} [[ref{idx+1}]]" for idx, doc in enumerate(docs)])
        ref_text = f"검색 결과:\n-----\n{ref_text}"
        tool_res = f"<tool_response>{ref_text}</tool_response>"
        messages.append({"role": "user", "content": tool_res})

        # (3) assistant 답변 (GPT로 RAG)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages,
            temperature=0
        )
        assistant_reply = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})

        # 다음 질문 생성 여부 랜덤 (5턴이 max면 4턴 미만일때만 질문 생성)
        if turn < max_turns - 1 and choice([True, False]):

            followup = question_chain.invoke({
                "history": messages
            })
            current_q = followup
        else:
            break

    return {"messages": messages}


In [ ]:
import time
from tqdm import tqdm

train_dataset = []
for q in tqdm(initial_questions):
    time.sleep(1)  # 1초 대기
    sample = generate_one_chat_sample(q, max_turns=4)
    train_dataset.append(sample)

# 저장
with open(f"qwen3_company_train_dataset_{permission}3.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, ensure_ascii=False, indent=2)


  0%|          | 0/295 [00:00<?, ?it/s]

내부 평가 피드백 문서에서 프론트엔드팀의 업무 수행 능력에 대한 성과는 어떤가요?
프론트엔드팀의 KPI 달성률이나 프로젝트 완료율에 대한 구체적인 수치도 확인할 수 있을까?
프론트엔드팀의 KPI 달성률이 낮았던 주요 원인이나 구체적인 개선 방안이 문서에 언급되어 있는지 알려줄 수 있어?


  0%|          | 1/295 [00:13<1:07:12, 13.72s/it]

프론트엔드팀의 팀워크 및 커뮤니케이션 부분에서 발생한 의견 충돌에 대한 구체적인 사례는 무엇인가요?


  1%|          | 2/295 [00:17<37:17,  7.64s/it]  

...

임원 인사·보상·승계 계획 문서에서 CTO의 관점에서 검증해야 할 포인트는 무엇인가요?


 67%|██████▋   | 197/295 [28:07<17:07, 10.48s/it]

임원 인사·보상·승계 계획 문서의 개요 및 목적에서 어떤 목표를 설정하고 있나요?
임원 인사·보상·승계 계획 문서에서 실제로 적용되는 보상 기준이나 승계 절차는 어떻게 정해져 있어?


 67%|██████▋   | 198/295 [28:19<17:24, 10.76s/it]

임원 인사·보상·승계 계획 문서에서 경쟁력 있는 보상 패키지 제공의 필요성은 어떻게 설명되고 있나요?
